# Smart restart

The idea of this tutorial is to show how to chain several computations while changing some elements of the pre-processing step in between them.

The chosen example is a shear of a 2D sample with thermo-rigids disks in a periodic channel. Because of the periodicity, it is more convenient to use a cluster of disks to generate the walls.
This computation is split into two computations:

1. First a purely mechanical computation with a vertical force to balance the sample
2. A second computation changing:
  * the material of the disks to get a thermo-rigid model
  * the boundary condition of the upper wall to shear
  
The point is to keep the exact state of the first computation, the contacts network included! To avoid any confusion, the two computations will be run in different directories named *Press* and *Shear* respectively.

## First sample generation

It is reminded that:

* the walls are cluster of disks
* the upper wall has a vertical force boundary condition
* the friction coefficient is null during this step
* there is no gravity

In [ ]:
from pathlib import Path
from pylmgc90 import pre

# creating directory 'Press/DATBOX' if not existing
datbox_path = Path('Press/DATBOX')
datbox_path.mkdir(parents=True, exist_ok=True)

dim = 2

#############
Friction          = 0.00
Friction_wall     = 0.00
#############
Rmin              = 0.05
Rmax              = 0.15

lx = 50*Rmax
ly = 10*Rmax
nb_particles      = 1000
#############

############## 
bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
svs    = pre.see_tables()
tacts  = pre.tact_behavs()

############## creating materials
# Note : these are in fact the same materials, just to make a difference between grains and walls
tdur = pre.material(name='TDURx',materialType='RIGID',density=2800.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=2800.)
mats.addMaterial(tdur,plex)

############## rigid model creation
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)
mods.addModel(mod)

############## granulo anb box building
radii  = pre.granulo_Uniform(nb_particles, Rmin, Rmax)

nb_remaining_particles, coor, radii = pre.depositInBox2D(radii, lx, ly)
############## loop adding particles:
for r, c in zip(radii, coor):
    body=pre.rigidDisk( model=mod, material=plex, center=c, r=r, color='BLUEx')
    #
    bodies += body

############## four walls creation with tdur material
down = pre.roughWall(center=[0.5*lx, -Rmin], l=lx, r=Rmin, model=mod, material=tdur, color='WALLx')
down.imposeDrivenDof(component=1, dofty='vlocy')
down.imposeDrivenDof(component=2, dofty='vlocy')
down.imposeDrivenDof(component=3, dofty='vlocy')
bodies += down

up   = pre.roughWall(center=[0.5*lx, ly+Rmin], l=lx, r=Rmin, model=mod, material=tdur, color='WALLx')
up.imposeDrivenDof(component=1, dofty='vlocy')
up.imposeDrivenDof(component=2, dofty='force',ct=-100000.,rampi=1.)
up.imposeDrivenDof(component=3, dofty='vlocy')
bodies += up

############## interactions management:
#   * law declarations
#       - particles vs particles and particles va walls
l1 = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=Friction)
tacts += l1
l2 = pre.tact_behav(name='iqsc2',law='IQS_CLB',fric=Friction_wall)
tacts += l2

#   * visibility tables declaration
#       - between particles of type (disk blue) vs (disk blue)
svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=l1,
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx', alert=Rmin)
svs += svdkdk
#       - between particles of type (disk blue) vs (disk wall)
svdkjc = pre.see_table(CorpsCandidat    ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx', behav=l2,
                        CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='WALLx', alert=Rmin)
svs += svdkjc


# file writing
post = pre.postpro_commands()
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox_path, gravy=[0.,0.,0.])

In [ ]:
pre.visuAvatars(bodies)

## First computation

The sample equilibrium computation under vertical loading. The computation is a classical one, except that:

1. the computation is run in the *Press* directory thanks to the `chipy.overall_SetWorkingDirectory` function,
2. the periodicity length is given thanks to the `chipy.SetPeriodicCondition`,
3. the `chipy.nlgs_SetWithQuickScramble` is used to accelerate solver convergence.


In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation

dim           = 2
Rmax          = 0.15
dt            = 1.e-4  # Time step
theta         = 0.5    # Time integrator
nb_steps      = 5000
freq_display  = 1000
freq_outFiles = 1000
freq_detect   = 1      # Contact detection frequency

h5_file       = 'lmgc90.h5'
#
#         123456789012345678901234567890
stype  = 'Stored_Delassus_Loops         '
norm   = 'QM/16'
tol    = 1e-4
relax  = 1.0
gs_it1 = 50                    # Minimum number of iterations
gs_it2 = 501                   # Maximum number of iterations  = gs_it2 * gs_it1
#
##############################
### INITIALIZE COMPUTATION ###
##############################
chipy.overall_SetWorkingDirectory('Press')
computation.initialize(dim, dt, theta, h5_file=h5_file, logmes=False)
chipy.SetPeriodicCondition(xperiod=50*Rmax)
chipy.nlgs_SetWithQuickScramble()

############################
### STARTING COMPUTATION ###
############################
for k in range(1, nb_steps+1):
    if k%500 == 0:
        print( f"computing step {k:4d}/{nb_steps}")
    computation.one_step(stype,norm,tol,relax,gs_it1,gs_it2,freq_outFiles,freq_display)

##########################
### END OF COMPUTATION ###
##########################

computation.finalize()

## Reading old computation to generate a new one

Now to prepare the second computation, it is desired to reload the result of this computation as *pre containers* before applying some changes before writing a new *DATBOX* directory.

The two functions to use are:
* `pre.readDatbox` to read a *DATBOX* directory and get the associated containers,
* `pre.readState` which will read a particular step of either a set of _OUTBOX/*.OUT_ files or an HDF5 file to change the configuration of the different avatars.

These two functions will also return a numpy array describing all interactions.

The changes to be made are to:
* change the friction coefficient
* add a velocity boundary condition along y-axis on the upper wall
* change the materials and models container to have a thermorigid

But first things first, let us start with loading the computation as *pre containers*:

In [ ]:
import fnmatch

from pylmgc90 import pre

dim = 2
mats, mods, bodies, tacts, sees, inters = pre.readDatbox(dim, str(datbox_path))
inters = pre.readState(bodies, 'Press/OUTBOX', -1, 'Press/lmgc90.h5', tacts)
#reading_step = len(fnmatch.filter(os.listdir('Press/OUTBOX/'), 'DOF.OUT.*'))
#inters = pre.readState(bodies, 'Press/OUTBOX', reading_step)

Now the containers should be in the same order than before writing the *Press* DATBOX but with the configuration of the computation. It is then possible to change the friction coefficient of the two contact laws.

In [ ]:
pre.visuAvatars(bodies)

In [ ]:
# change friction parameters of contact laws
for k, v in tacts.items():
    print( f"updating friction parameters of law {k}" )
    v.fric = 0.2

Sometimes it is not as straightforward to identify some specific bodies. For example, remembering that the walls have the *WALLx* color, it is really easy to find the two wall avatars. But to know which one is the upper or lower one, either a check on the avatar coordinates, or just remembering in which order they have been added. Once the avatar identify there is no difference with regular boundary condition addition.

In [ ]:
# look for wall avatars by color:
walls = []
for b in bodies:
    if b.contactors[0].color == 'WALLx':
        walls.append(b)

# get the upper wall:
upper = walls[0] if walls[0].getNodeCoor()[1] > walls[1].getNodeCoor()[1] else walls[1]

# assert that the upper wall is really the second one
assert upper is walls[1], "something strange when looking for upper wall"

# adding boundary conditions in velocity
# without changing the existing previous ones on down and up
upper.imposeDrivenDof(component=1, dofty='vlocy',ct=1.,rampi=1.)

Finally the last bit, a little more technical, is to write the thermorigid models. Currently the thermorigid model and material writing is made thanks to external script provided within the current directory of this notebook (*MP_mat.py* and *MP_mod.py*). Thus the logic is:
* to reset the material container to be empty,
* to write all the avatar containers as usual,
* then to write the thermorigid model and materials using the provided scripts.

**Beware:** the name of the model and material of each avatar is already defined and identified by a five character string inside each objects. Thus when creating the model and materials, they must be have the same name ! Otherwise there will be error when trying to initialize LMGC90 database when starting the computation part.

In [ ]:
wall_mat = walls[0].bulks[0].material.nom
disk_mat = bodies[0].bulks[0].material.nom
print( f"material name of walls: {wall_mat}")
print( f"material name of disks: {disk_mat}")

In [ ]:
# reset materials to use MP ones
mats = pre.materials()

# creating new directory in which to write DATBOX
datbox_path = Path('Shear/DATBOX')
datbox_path.mkdir(parents=True, exist_ok=True)

# write DATBOX in Shear directory
post = pre.postpro_commands()
pre.writeDatbox(dim, mats, mods, bodies, tacts, sees, inters, post=post, datbox_path=datbox_path, gravy=[0.,0.,0.])

# create new thermorigid model/materials
import MP_mat
import MP_mod

#MP_mod.write_thermal_model(T0=0.23e+2,alert=rmax,ldiff='Cylnd',gdiff='discrete',lconv='no',lkine='all',bound='1D__')

rmax=0.5
lx = 50*rmax

MP_mod.write_thermal_model(T0=0.23e+2, alert=rmax, ldiff='Cylnd', gdiff='discrete',
                           lconv='no', lkine='all', bound='adia', PATH=str(datbox_path))

#MP_mod.write_thermal_source(first=down.number+1,last=down.number+1,T0=100.)
#MP_mod.write_thermal_bounds(first=up.number+1,last=up.number+1,T0=1000.,thickness=4*lx,length=2*lx,alpha=1e-1,locus='U')

# wall
MP_mat.write_thermal_material(name=wall_mat,materialType='THERMO_RIGID',density=7800.,
                              anisotropy='isotropic',thermal_conductivity=3.9e+02,specific_heat=3.8e+02,
                              thermal_young=1.24e+09,thermal_nu=0.3e+00,PATH=str(datbox_path))
# particles
MP_mat.write_thermal_material(name=disk_mat,materialType='THERMO_RIGID',density=7800.,
                              anisotropy='isotropic',thermal_conductivity=3.9e+02,specific_heat=3.8e+02,
                              thermal_young=1.24e+09,thermal_nu=0.3e+00,PATH=str(datbox_path))

## Second computation

Now a thermorigid computation can be made. The *computations* macro cannot be used in this case and a more old fashionned script content is presented here. The computation still make uses of:

* the `chipy.overall_SetWorkingDirectory` function to work in the *Shear* directory,
* the `chipy.SetPeriodicCondition to set` the periodicity of the sample,
* the `chipy.nlgs_SetWithQuickScramble` to accelerate solver convergence,
* the `chipy.RBDY2_FatalDamping`.

Furthermore to use the *thermorigid* feature the following functions must also be used:

* `chipy.ReadMpBehaviours` during initialization to read the files,
* `chipy.mp_solver_RecupTemperature` and `chipy.mp_solver_SolveThermoProblem` after the contact resolution to solve the thermal part.

Finally, the *temperature* field is added to the visualization files. This needs several code blocks along the simulation.

In [ ]:
import numpy as np
from pylmgc90 import chipy

chipy.Initialize()
chipy.overall_SetWorkingDirectory('Shear')

#############
# Parameters
#############
#
# ... of the simulation
nb_steps    = 10000
dt          = 5.e-4  # Time step
theta       = 0.5    # Time integrator
freq_detect = 1      # Contact detection frequency
Rmax        = 0.15
freq_visu   = 100    # Number of visualization files
freq_write  = 100    # Number of outputs in file

h5_file     = 'lmgc90.h5'

#
# ... of the computation
#         123456789012345678901234567890
stype  = 'Stored_Delassus_Loops         '
norm   = 'QM/16'
tol    = 1e-5
relax  = 1.0
gs_it1 = 100                   # Minimum number of iterations
gs_it2 = 101                   # Maximum number of iterations = gs_it2 * gs_it1
chipy.nlgs_SetWithQuickScramble()
#
#############
chipy.utilities_DisableLogMes()      # Log message management
#chipy.utilities_EnableLogMes()
chipy.checkDirectories()             # Check if all subdirectories are presents
chipy.SetDimension(2)                # Set dimension in chipy for dummies  ###

#####################
### Model reading ###
#####################

chipy.utilities_logMes('READ BODIES')
chipy.ReadBodies()

chipy.utilities_logMes('READ BEHAVIOURS')
chipy.ReadBehaviours()

chipy.utilities_logMes('LOAD BEHAVIOURS')
chipy.LoadBehaviours()

chipy.utilities_logMes('LOAD TACTORS')
chipy.LoadTactors()

# THERMO RIGID
chipy.ReadMpBehaviours(0,'therm')

chipy.ReadIni()
#
chipy.utilities_logMes('READ DRIVEN DOF')
chipy.ReadDrivenDof()
############################
### End of Model reading ###
############################

chipy.SetPeriodicCondition(xperiod=50*Rmax)

#########################################
### Computation parameters definition ###
#########################################
chipy.utilities_logMes('INIT TIME STEPPING')
chipy.TimeEvolution_SetTimeStep(dt)
chipy.Integrator_InitTheta(theta)

### Init postpro ###
chipy.OpenDisplayFiles()
chipy.OpenPostproFiles()
chipy.InitHDF5(h5_file)

### COMPUTE MASS ###
chipy.ComputeMass()

############################
### STARTING COMPUTATION ###
############################

nbdiskx     = chipy.DISKx_GetNbDISKx()
diskx2rbdy2 = chipy.DISKx_GetPtrDISKx2BDYTY()

nbjoncx     = chipy.JONCx_GetNbJONCx()
joncx2rbdy2 = chipy.JONCx_GetPtrJONCx2BDYTY()

nbR2        = chipy.RBDY2_GetNbRBDY2()

Th_DISKx = np.zeros([nbdiskx])
Th_JONCx = np.zeros([nbjoncx])

T_dict = {'DISKx':Th_DISKx, 'JONCx':Th_JONCx}


chipy.RBDY2_FatalDamping()

for k in range(1, nb_steps+1):
    #
    if k%500 == 0:
        print( f"computing step {k:4d}/{nb_steps}")

    chipy.IncrementStep()
    #
    chipy.ComputeFext()
    #
    chipy.ComputeBulk()
    chipy.ComputeFreeVelocity()
    #
    chipy.SelectProxTactors(freq_detect)
    #
    chipy.RecupRloc()
    chipy.nlgs_ExSolver(stype,norm,tol,relax,gs_it1,gs_it2)
    chipy.StockRloc()
    #
    # THERMO RIGID
    #
    chipy.mp_solver_RecupTemperature()
    chipy.mp_solver_SolveThermoProblem()

    #
    chipy.ComputeDof()
    chipy.UpdateStep()
    #
    chipy.WriteOut(freq_write)
    #
    # THERMO RIGID
    #
    chipy.WriteOutMpValues(freq_write)
    #
    ### postpro ###
    if (k % freq_visu == 0 ):
        #
        for itacty in range(1,nbdiskx+1,1):
            iddiskx = diskx2rbdy2[itacty-1]
            idR2 = int(iddiskx[0])
            idTy = int(iddiskx[1])
            # THERMO RIGID
            Th_DISKx[itacty-1] = chipy.RBDY2_GetThermalValue(idR2,idTy)
         
        for itacty in range(1,nbjoncx+1,1):
            idjoncx = joncx2rbdy2[itacty-1]
            idR2 = int(idjoncx[0])
            idTy = int(idjoncx[1])
            # THERMO RIGID
            Th_JONCx[itacty-1] = chipy.RBDY2_GetThermalValue(idR2,idTy)

        chipy.WriteDisplayFiles(1,T =('tacts',T_dict))
     
    chipy.WritePostproFiles()

    chipy.checkInteractiveCommand()

##########################
### END OF COMPUTATION ###
##########################

chipy.CloseDisplayFiles()
chipy.ClosePostproFiles()
chipy.Finalize()

In [ ]:
!paraview